In [1]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt 
from wordcloud import wordcloud 
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amitj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amitj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\amitj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df =pd.read_csv('spam_or_not_spam.csv')

In [4]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [5]:
from sklearn.preprocessing import LabelEncoder

encoder =LabelEncoder()
df['label']=encoder.fit_transform(df['label'])

In [6]:
df.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [7]:
df.duplicated().sum()

127

In [8]:
len(df)

3000

Feature Enginnering

In [9]:
from nltk.stem.porter import PorterStemmer

import string 


ps =PorterStemmer()




In [10]:
def transform_text(text):
    if not isinstance(text, str):
        return ""  # or return np.nan if you prefer to mark missing data
    
    text = text.lower()
    text = nltk.word_tokenize(text)
    
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)
    
    text = y[:]
    y.clear()
    
    for i in text:
        y.append(ps.stem(i))
        
    return " ".join(y)


In [11]:
transform_text(' trth oikfgg kl.d.. ioj toiotr4%  ykorhmgf,jik8* jiuyettetkjehiw22hkjbjertjrijryi53i4534')

'trth oikfgg ioj toiotr4 ykorhmgf jik8 jiuyettetkjehiw22hkjbjertjrijryi53i4534'

In [12]:
df['transformed_text']=df['email'].apply(transform_text)

In [13]:
df.head()

,email,label,transformed_text
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,date wed number aug number number number numbe...
1,martin a posted tassos papadopoulos the greek ...,0,martin post tasso papadopoulo greek sculptor b...
2,man threatens explosion in moscow thursday aug...,0,man threaten explos moscow thursday august num...
3,klez the virus that won t die already the most...,0,klez viru die alreadi prolif viru ever klez co...
4,in adding cream to spaghetti carbonara which ...,0,ad cream spaghetti carbonara effect pasta make...


In [14]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
tfid =TfidfVectorizer(max_features =500)

In [15]:
x = tfid.fit_transform(df['transformed_text']).toarray()
y = df['label'].values

Train Test Split

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test , y_train ,y_test = train_test_split(x,y,test_size=0.20 ,random_state =2)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB 
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [18]:
svc =SVC(kernel="sigmoid",gamma = 1.0)
knc =KNeighborsClassifier()
mnb =MultinomialNB()
dtc =DecisionTreeClassifier()
irc = LogisticRegression(solver ='liblinear' , penalty='l1')
rfc = RandomForestClassifier(n_estimators=50 , random_state=2)
abc = AdaBoostClassifier(n_estimators=50,random_state=2)
bc = BaggingClassifier(n_estimators=50 ,random_state=2)
etc = ExtraTreesClassifier(n_estimators=50 , random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50 , random_state=2)
xgb = XGBClassifier(n_estimators =50, random_state =2)

In [23]:
models ={
    'SVC':svc,
    'KNN':knc,
    'NB':mnb,
    'DI':dtc,
    'LOG':irc,
    'RF':rfc,
    'AD':abc,
    'BC':bc,
    'EX':etc,
    'GB':gbdt,
    'XGB':xgb
}

In [24]:
from sklearn.metrics import accuracy_score , precision_score
def train_classifier(model ,x_train , y_train ,x_test ,y_test):
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision =precision_score(y_test, y_pred)
    return accuracy , precision

In [25]:
accuracy_scores = []
precision_scores = []

for name, model in models.items():
    current_accuracy, current_precision = train_classifier(model, x_train, y_train, x_test, y_test)  # <-- model not models
    print()
    print("For", name)
    print("Accuracy:", current_accuracy)
    print("Precision:", current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)



For SVC
Accuracy: 0.9683333333333334
Precision: 0.9444444444444444

For KNN
Accuracy: 0.7133333333333334
Precision: 0.3545816733067729

For NB
Accuracy: 0.9733333333333334
Precision: 1.0

For DI
Accuracy: 0.9466666666666667
Precision: 0.8252427184466019

For LOG
Accuracy: 0.9716666666666667
Precision: 1.0

For RF
Accuracy: 0.9766666666666667
Precision: 0.9885057471264368

For AD
Accuracy: 0.9683333333333334
Precision: 0.9761904761904762

For BC
Accuracy: 0.9583333333333334
Precision: 0.8854166666666666

For EX
Accuracy: 0.9783333333333334
Precision: 0.9886363636363636

For GB
Accuracy: 0.965
Precision: 0.9875

For XGB
Accuracy: 0.9766666666666667
Precision: 0.9885057471264368


In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
# Evaluate all models
for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    
    print(f"----- {name} -----")
    print("Accuracy :", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, average='weighted'))
    print("Recall   :", recall_score(y_test, y_pred, average='weighted'))
    print("F1 Score :", f1_score(y_test, y_pred, average='weighted'))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("\n\n")


----- SVC -----
Accuracy : 0.9683333333333334
Precision: 0.9679117647058824
Recall   : 0.9683333333333334
F1 Score : 0.9677203146342612
Confusion Matrix:
 [[496   5]
 [ 14  85]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       501
           1       0.94      0.86      0.90        99

    accuracy                           0.97       600
   macro avg       0.96      0.92      0.94       600
weighted avg       0.97      0.97      0.97       600




----- KNN -----
Accuracy : 0.7133333333333334
Precision: 0.8695804746629529
Recall   : 0.7133333333333334
F1 Score : 0.7499495798319328
Confusion Matrix:
 [[339 162]
 [ 10  89]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.68      0.80       501
           1       0.35      0.90      0.51        99

    accuracy                           0.71       600
   macro avg       0.66      0.79      0.65     

In [27]:
df.head()

,email,label,transformed_text
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0,date wed number aug number number number numbe...
1,martin a posted tassos papadopoulos the greek ...,0,martin post tasso papadopoulo greek sculptor b...
2,man threatens explosion in moscow thursday aug...,0,man threaten explos moscow thursday august num...
3,klez the virus that won t die already the most...,0,klez viru die alreadi prolif viru ever klez co...
4,in adding cream to spaghetti carbonara which ...,0,ad cream spaghetti carbonara effect pasta make...


In [29]:

from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
tfid =TfidfVectorizer(max_features =500)
tfid.fit(df['transformed_text'])

x = tfid.transform(df['transformed_text']).toarray()

# Suppose 'best_model' is the model you selected
best_model = rfc  # Example: using XGBoost (you can change to your best model)

# Now predict for a new email
def predict_email(email_text):
    # Step 1: Transform text
    transformed_email = transform_text(email_text)
    
    # Step 2: Vectorize (assuming you used TF-IDF or CountVectorizer)
    vector_input = tfid.transform([transformed_email])  # tfidf should be your trained vectorizer
    
    # Step 3: Predict
    result = best_model.predict(vector_input)[0]
    
    # Step 4: Return the result
    if result == 1:
        return "Spam 🚫"
    else:
        return "Not Spam ✅"

# Example use
email = "Congratulations! You've won a $1000 Walmart gift card. Click here to claim now."
print(predict_email(email))

email2 = "Hey, are we still on for the meeting tomorrow morning?"
print(predict_email(email2))



Spam 🚫
Spam 🚫
